<a href="https://colab.research.google.com/github/HugoTHO/sisbi-utilities-notebooks/blob/main/sigaa_register_daily_movement_of_users.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AUTOMAÇÃO DE REGISTRO DE MOVIMENTAÇÃO DIÁRIA DE USUÁRIOS
***

## Importação de bibliotecas Python

In [1]:
import bs4
import json
import re
import requests
import urllib.parse as urlparser
import calendar

from datetime import datetime, timedelta
from getpass import getpass
from ipywidgets import widgets
from IPython.display import display, HTML
from html import unescape

## Login

In [ ]:
#@markdown Execute esta célula e forneça sua credencial do Sigaa e senha.
#@markdown Em seguida, escolha seu vínculo. <hr>
#@markdown O login no sistema e escolha do vínculo são necessários para as 
#@markdown próximas operações.

base_url = "https://sigaa.ufrn.br"
base_path = "/sigaa/"
relship_sel_path = "/sigaa/vinculos.jsf"

# INTERFACE UTILITIES
dropbox_placeholder = "SELECIONE O VINCULO"
default_box_lt = widgets.Layout(padding='0 0 28px 0', border='1px solid white')
success_box_lt = widgets.Layout(padding='0 0 28px 0', border='1px solid green')

session = requests.Session()
login_page = session.get(urlparser.urljoin(base_url, base_path))

# Checks if a valid user is already logged
try:
    username
except NameError:
    username = input("Usuário: ")
    password = getpass("Senha: ")

# Mounts the POST necessary login data
login_form = (
    bs4.BeautifulSoup(login_page.text)
    .find('form', {'id':'login-form'})
)
login_path = login_form['action']
login_data = {
    'username':username,
    'password':password
}
for hidden_input in login_form.find_all('input',{'type':'hidden'}):
    login_data[hidden_input['name']] = hidden_input['value']

response = session.post(
    urlparser.urljoin(login_page.url,login_path),
    data=login_data
)

if len(response.history) and response.history[0].status_code > 300:
    print('Login realizado com sucesso, escolha seu vínculo:')

    relationship_page = session.get(
        urlparser.urljoin(base_url, relship_sel_path)
    )
    relship_links = (
        bs4.BeautifulSoup(relationship_page.text)
        .find_all('a', class_='withoutFormat')
    )

    # Concatenates the relationship type and description on link list,
    # converting the escaped charactes.
    choices = [
        (reltype.string + 
        unescape(relship.string.split(':')[1]
            .encode('ascii', 'xmlcharrefreplace')
            .decode('utf-8'))
        )
        for reltype, relship in zip(
            relship_links[::3],
            relship_links[2::3]
        )
    ]

    relship_dd = widgets.Dropdown(
        options=[dropbox_placeholder] + choices,
        value=dropbox_placeholder,
        layout=default_box_lt
    )
    display(relship_dd)

    # Registers a handler function to access the choiced relationship
    def dd_change(change):
        if (change['new'] != 0):
            relship_path = relship_links[(change['new']-1)*3]['href']
            session.get(urlparser.urljoin(base_url, relship_path))
            relship_dd.layout = success_box_lt
        else:
            relship_dd.layout = default_box_lt

    relship_dd.observe(dd_change, names='index')

else:
    print('Problema ao realizar o login')
    # deletes the username variable to reinsert the credentials
    del username

## Consultar e cadastrar movimentação diária

In [ ]:
#@title Escolher biblioteca, mês e ano
#@markdown <hr> 
#@markdown Execute essa célula paara visualizar e escolher entre as opções
#@markdown disponíveis

library_path = "/sigaa/biblioteca/index.jsf"
page_link_str = "Cadastrar Movimentação Diária de Usuários"

month_list = ["Janeiro", "Fevereiro", "Março", "Abril",
              "Maio", "Junho", "Julho", "Agosto",
              "Setembro", "Outubro", "Novembro", "Dezembro"]


def get_jsfcl_path(command_link):
    '''Given a JSF command_link bs4.tag, returns path to access 
    the next page.
    '''
    return command_link.find_parent('form')['action']

def get_jsfcl_json(command_link):
    '''Given a JSF command_link bs4.tag, returns json content
    of jsfcls() javascript junction call.
    '''
    # Gets the jsfcljs javascript function's json param on JSF command link's
    # onclick attr that references next page.
    #
    # more info:
    # https://jakarta.ee/specifications/faces/2.3/jsdoc/jsf-uncompressed.js.html
    re_pattern_str = r"""jsfcljs\(                            # function call
                         (?P<form>.+?),                       # form param
                         (?P<json>{(?:(?:.+?):(?:.+?),?)+?}), # json param
                         (?:(?P<target>.+?)\))                # target param"""
    return json.loads(
        re.search(
            pattern=re_pattern_str,
            string=command_link['onclick'],
            flags=re.VERBOSE,
        ).group('json').replace("'",'"')
    )

def get_jsfcl_post_data(command_link):
    '''Given a JSF command_link bs4.tag, returns the post data to 
    access the next page.
    '''
    post_data = { tag['name']: tag['value'] 
        if tag.has_attr('value') 
        else ""
        for tag in (command_link
            .find_parent('form')
            .find_all('input', type=lambda t: t not in ['submit','radio']))
    }

    post_data.update(get_jsfcl_json(command_link))
    
    return post_data


def display_options(choices_dict):
    '''TODO: document this function
    '''
    choice_wgets = {}
    # Basic Widgets
    choice_wgets['month'] = widgets.Dropdown(
        options = month_list,
        value = month_list[int(choices_dict['month']['value'])-1]
    )
    choice_wgets['year'] = widgets.IntText(choices_dict['year']['value'])
    choice_wgets['library'] = widgets.Dropdown(
        options = [option['string']
            for option in choices_dict['library']['options']
        ],
        value = choices_dict['library']['options'][0]['string']
    )

    # Containers
    label_vb = widgets.VBox([
        widgets.Label("Mês da Frequência:"),
        widgets.Label("Ano da Frequência:"),
        widgets.Label("Biblioteca:")
    ])
    widgets_vb = widgets.VBox([
        choice_wgets['month'],
        choice_wgets['year'],
        choice_wgets['library']
    ])

    display(widgets.HBox([label_vb, widgets_vb]))
    return choice_wgets


library_page = session.get(urlparser.urljoin(base_url, library_path))
link_tag = (
    bs4.BeautifulSoup(library_page.text)
    .find('a', string=page_link_str)
)

lib_month_year_page = session.post(
    urlparser.urljoin(base_url, get_jsfcl_path(link_tag)),
    data=get_jsfcl_post_data(link_tag)
)

options_form = (bs4.BeautifulSoup(lib_month_year_page.text)
    .find('form', id='formulario'))

form_data = {}

library_choices = None
for form_field in (options_form.find_all('th')):
    if 'Mês' in form_field.string:
        form_data['month'] = {
            'name': form_field.findNext('input')['name'],
            'value': form_field.findNext('input')['value']
        }
    elif 'Ano' in form_field.string:
        form_data['year'] = {
            'name': form_field.findNext('input')['name'],
            'value': form_field.findNext('input')['value']
        }
    else:
        form_data['library'] = {
            'name': form_field.find_next('select')['name'],
            'options': [{
                'string': option.string,
                'value': option['value']
                } for option in (form_field
                    .find_next('select')
                    .find_all('option')
                )
            ]
        }

choice_wgets = display_options(form_data)

In [ ]:
#@title Mostar formulário de registro
submit_value = 'Consultar'

form_style = """<style>
    div.int-input{
        width:80px
    }
    div.started-input > input[type=number] {
        border: 1px solid #64A7A7;
        background-color: #F3FDFD;
    }
    div.day-box, div.label-box{
        width:130px;
        border: 1px solid #CCC;
        margin: 1px 1px 1px 1px;
    }
    div.blank-box{
        background-color: #FAFAFA;
    }
    div.label-box{
        border: 1px solid #888;
        background-color: #EEE;
    }
    div.centered-box{
        display: flex;
        justify-content: center;
    }
    </style>"""


def get_holydays(year):
    ''' Determines the holidays, given a year
    '''
    holydays = {
        1: {
            1: "Confraternização Universal",
            6: "Dia de Santos Reis"
        },
        2: {},
        3: {},
        4: {21: "Dia de Tiradentes"},
        5: {1: "Dia Mundial do Trabalho"},
        6: {24: "Dia de São João"},
        9: {7: "Dia da Independência do Brasil"},
        10: {
            3: "Dia dos Mártires de Cunhaú e Uruaçu",
            12: "Dia de Nossa Senhora Aparecida",
            28: "Dia do Servidor Público"
        },
        11: {
            2: "Dia de Finados",
            15: "Dia da Proclamação da República",
            21: "Dia de Nossa Senhora da Apresentação"
        },
        12: {
            24: "Véspera de Natal",
            25: "Natal",
            31: "Véspera de Ano Novo"
        }
    }
    
    # Includes Easter, Carnival and Corpus Christi to holydays
    easter_sum = (((19 * (year % 19) + 24) % 30)
        + ((2 * (year % 4)
        + 4 * (year % 7)
        + 6 * ((19 * (year % 19) + 24) % 30)
        + 5)
        % 7))
    if easter_sum < 10:
        easter_day = easter_sum + 22
        easter_month = 3
    else:
        easter_day = easter_sum - 9
        easter_month = 4
        
    easter_dt = datetime(year=year, month=easter_month, day=easter_day)
    carnival_dt = easter_dt - timedelta(47)
    corpus_christi_dt = easter_dt + timedelta(60)
    
    holydays[carnival_dt.month][carnival_dt.day - 1] = "2ª de Carnaval"
    holydays[carnival_dt.month][carnival_dt.day] = "3ª de Carnaval"
    holydays[carnival_dt.month][carnival_dt.day + 1] = "4ª de Cinzas"
    holydays[easter_dt.month][easter_dt.day - 3] = "5ª Santa"
    holydays[easter_dt.month][easter_dt.day - 2] = "6ª Santa"
    holydays[corpus_christi_dt.month][corpus_christi_dt.day] = "Corpus Christi"

    return holydays

def generate_grade(form, year, month, form_dict, reg_dict):

    # -- Weekdays labels
    weekdays = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta"]
    wd_labels = [ widgets.Box([widgets.Label(name)])
        for name in weekdays
    ]
    for label_box in wd_labels:
        label_box.add_class('label-box')
        label_box.add_class('centered-box')
    form.append(widgets.HBox(wd_labels))

    # build the form elements
    weekday_list = []
    
    # -- Holydays
    holydays = get_holydays(year)

    # Fill form
    first_week_day, last_day = calendar.monthrange(year, month)
    
    # month starts in any other day of the workweek than Monday...
    if first_week_day > 0 and first_week_day < 5:
        for week_day in range(first_week_day):
            blank_box = widgets.Box([])
            blank_box.add_class('blank-box')
            weekday_list.append(blank_box)
    
    for day in range(1, last_day+1):
        weekday = datetime(
            year=year,
            month=month,
            day=day
        ).isoweekday() # monday = 1, tuesday = 2, ...

        if weekday < 6:
            # New week and week_list isn't empty
            if weekday == 1 and day != 1 and len(weekday_list):
                form.append(widgets.HBox(weekday_list))
                weekday_list = []
            
            if month in holydays and day in holydays[month]:
                holyday_box = widgets.Box([
                    widgets.Label(holydays[month][day])
                ])
                holyday_box.add_class('centered-box')
                blank_box = widgets.VBox([
                    widgets.Label("{:02d}".format(day)),
                    holyday_box
                ])
                blank_box.add_class('blank-box')
                weekday_list.append(blank_box)
            else:
                # Create shift inputs
                shift_inputs = [
                    widgets.IntText(),
                    widgets.IntText(),
                    widgets.IntText(),
                ]

                if day in reg_dict:
                    for i, input in enumerate(shift_inputs):
                        if i in reg_dict[day]:
                            input.set_trait(
                                name='value',
                                value=reg_dict[day][i]['count']
                            )
                            input.add_class('started-input')
                
                for w in shift_inputs:
                    w.add_class('int-input')
                
                # Append day
                weekday_list.append(
                    widgets.HBox([
                        widgets.Label("{:02d}".format(day)),
                        widgets.VBox([
                            widgets.Label("M:"),
                            widgets.Label("T:"),
                            widgets.Label("N:")
                        ]),
                        widgets.VBox(shift_inputs)
                    ])
                )
                
                # Append references
                form_dict[day] = shift_inputs

    
    # Append last week
    form.append(widgets.HBox(weekday_list))

    # Adds class to style
    for child in form[1:]:
        for grandchild in child.children:
            grandchild.add_class('day-box')


def generate_form(form_box, form_dict, reg_dict):
    # Removes last form children
    if len(form_box.children) > 0:
        form_box.children = []
        form_dict.clear()
    
    form = []

    # Add week grade
    selected_year = choice_wgets['year'].value
    selected_month = choice_wgets['month'].index + 1
    generate_grade(form, selected_year, selected_month, form_dict, reg_dict)
    
    # Update the formulary box
    form_box.children = form


def fill_registry_dict(registry_page):
    reg_dict = {}
    try:
        # scrapes the movement data to reg_dict dictionary
        mv_reg_entries = (bs4.BeautifulSoup(registry_page.text)
            .find('table', class_='subFormulario')
            .findChild('table')
            .find_all('td'))

        for i in range(0, len(mv_reg_entries), 5):
            day = int(mv_reg_entries[i].string[:2])
            
            shift_initial = mv_reg_entries[i+1].string[:1]
            shift = ['M','V','N'].index(shift_initial)
            
            if day not in reg_dict:
                reg_dict[day] = {}
            
            reg_dict[day][shift] = {
                'count': int(mv_reg_entries[i+2].string),
                'remove_command': mv_reg_entries[i+4].findChild('a')
        }
    except AttributeError:
        # There aren't any entry in selected month
        pass
    finally:
        return reg_dict

post_data = {input['name']: input['value'] 
    for input in options_form.find_all('input', type='hidden')
}
post_data[options_form.find(
    name='input',
    value=submit_value
)['name']] = submit_value

# TODO Comment explaining this block
post_data[form_data['library']['name']] = (form_data
    .get('library')
    .get('options')[choice_wgets['library'].index]
    .get('value')
)
selected_month = choice_wgets['month'].index + 1
selected_year = choice_wgets['year'].value
post_data[form_data['month']['name']] = selected_month
post_data[form_data['year']['name']] = selected_year
    
movement_register_page = session.post(
    urlparser.urljoin(base_url, options_form['action']),
    data=post_data
)

mv_reg_dict = fill_registry_dict(movement_register_page)

form_dict = {}
form_box = widgets.VBox([])
generate_form(form_box, form_dict, mv_reg_dict)

display(HTML(form_style))
display(form_box)

In [83]:
#@title Cadastrar Frequência

submit_value = 'Cadastrar'

register_form = (bs4.BeautifulSoup(movement_register_page.text)
    .find('form', {'name':'form'})
)

post_data = {}

library_choiced = register_form.find(name='option', selected='selected')
post_data[library_choiced.find_parent()['name']] = library_choiced['value']

submit_input = register_form.find(name='input', value=submit_value)
post_data[submit_input['name']] = submit_input['value']

for day, inputs in form_dict.items():
    for shift, input in enumerate(inputs, 1):
        registered = 'started-input' in input._dom_classes
        if not registered and  input.value: # not zero
            shift_radio = (register_form
                .find('input', type='radio', value=str(shift))
            )
            post_data[shift_radio['name']] = shift_radio['value']

            date_input = register_form.find(
                name='input',
                type='text',
                maxlength='10'
            )
            post_data[date_input['name']] = (datetime(
                    year=selected_year,
                    month=selected_month,
                    day=day
                ).strftime('%d/%m/%Y')
            )

            count_input = register_form.find(
                name='input',
                type='text',
                maxlength='4'
            )
            post_data[count_input['name']] = str(input.value)

            post_data.update({
                input['name']: input['value'] 
                for input in register_form.find_all('input', type='hidden')
            })

            movement_register_page = session.post(
                urlparser.urljoin(base_url, register_form['action'])
                ,data=post_data
            )

            if movement_register_page.ok:
                input.add_class('started-input')
                register_form = (bs4.BeautifulSoup(movement_register_page.text)
                    .find('form', {'name':'form'})
                )
            else:
                raise Exception(
                    'post request failure, day:{} shift:{}'.format(day, shift)
                )